In [1]:
#@title Setup for Colab only
#!pip install transformers
#!pip install git+git://github.com/corolla-johnson/mkultra.git#egg=mkultra

In [2]:
from transformers.pipelines import pipeline
from mkultra.models.tuning import GPT2PromptTuningLM
from mkultra.tokenizers import GPT2TokenizerFast
from mkultra.soft_prompt import SoftPrompt
from mkultra.trainers import SoftPromptTrainer

In [3]:
# Use an mkultra prompt tuning LM and a standard tokenizer.
model = GPT2PromptTuningLM.from_pretrained("gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [1]:
basic_prompt = "The sky over the port was the color of television tuned to a" #@param {type:"string"}

ids = tokenizer(basic_prompt, return_tensors="pt").input_ids
basic_output = model.generate(
    ids,
    do_sample=True,
    min_length=ids.shape[-1] + 100,
    max_length=ids.shape[-1] + 100,
    temperature=0.7,
    top_p = 0.9,
    repetition_penalty = 1.0,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(basic_output[0]))

NameError: name 'tokenizer' is not defined